In [1]:
import requests
from datetime import datetime
import time
import pandas as pd
from tqdm import tqdm

In [2]:
#전체 종목 불러오기
result = requests.get('https://api.binance.com/api/v3/ticker/price')
js = result.json()
symbols = [x['symbol'] for x in js]
symbols_usdt = [x for x in symbols if 'USDT' in x]  # 끝이 USDT로 끝나는 심볼들, ['BTCUSDT', 'ETHUSDT', ...]

In [3]:
#CSV 파일 만들기 , 여기서는 종목 1개로 진행한다.
COLUMNS = ['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time', 'quote_av', 'trades',
                   'tb_base_av', 'tb_quote_av', 'ignore']
URL = 'https://api.binance.com/api/v3/klines'#캔들스틱정보를 가져오는 api 주소
def get_data(start_date, end_date, symbol):
    data = []
##날짜 변환
    start = int(time.mktime(datetime.strptime(start_date + ' 00:00', '%Y-%m-%d %H:%M').timetuple())) * 1000
    end = int(time.mktime(datetime.strptime(end_date +' 23:59', '%Y-%m-%d %H:%M').timetuple())) * 1000
    params = {
        'symbol': symbol,
        'interval': '5m',
        'limit': 1000,
        'startTime': start,
        'endTime': end
    } 

    while start < end:
        print(datetime.fromtimestamp(start // 1000))
        params['startTime'] = start
        result = requests.get(URL, params = params)
        js = result.json()
        if not js:
            break
        data.extend(js)  # result에 저장
        start = js[-1][0] + 60000  # 다음 step으로
    # 전처리
    if not data:  # 해당 기간에 데이터가 없는 경우
        print('해당 기간에 일치하는 데이터가 없습니다.')
        return -1
    df = pd.DataFrame(data)
    df.columns = COLUMNS
    df['Open_time'] = df.apply(lambda x:datetime.fromtimestamp(x['Open_time'] // 1000), axis=1)
    df = df.drop(columns = ['Close_time', 'ignore'])
    df['Symbol'] = symbol
    df.loc[:, 'Open':'tb_quote_av'] = df.loc[:, 'Open':'tb_quote_av'].astype(float)  # string to float
    df['trades'] = df['trades'].astype(int)
    return df

start_date = '2021-11-01'
end_date = '2022-01-12'
symbol = symbols_usdt[0]
df = get_data(start_date, end_date, symbol)
df.to_csv(f'C:경로/{symbol[:3].lower()}_20220113.csv', index=False)

2021-11-01 00:00:00
2021-11-04 11:16:00
2021-11-07 22:36:00
2021-11-11 09:56:00
2021-11-14 21:16:00
2021-11-18 08:36:00
2021-11-21 19:56:00
2021-11-25 07:16:00
2021-11-28 18:36:00
2021-12-02 05:56:00
2021-12-05 17:16:00
2021-12-09 04:36:00
2021-12-12 15:56:00
2021-12-16 03:16:00
2021-12-19 14:36:00
2021-12-23 01:56:00
2021-12-26 13:16:00
2021-12-30 00:36:00
2022-01-02 11:56:00
2022-01-05 23:16:00
2022-01-09 10:36:00
2022-01-12 21:56:00
2022-01-12 23:56:00
